### 3 · Regularized MMM with Adstock & Saturation (Meridian-free)

In [6]:
import pandas as pd, numpy as np
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy.special import expit
from scipy.optimize import minimize

df = pd.read_csv("../data/marketing_data_clean.csv")
channels = ["tv_spend","search_spend","social_spend","display_spend"]
controls = ["price_index","promo","holiday"]

def adstock(x, L=8, alpha=0.6):
    w = alpha ** np.arange(L)
    w = w / w.sum()
    return np.convolve(x, w, mode="full")[:len(x)]

def saturation(x, k=1.0, s=0.5): 
    return (x ** s) / (x ** s + k ** s)

X_list = []
for c in channels:
    xs = adstock(df[c].values, L=8, alpha=0.6)
    xs = saturation(xs, k=np.quantile(xs, 0.7) + 1e-6, s=0.6)
    X_list.append(xs)
X = np.vstack(X_list).T

Z = df[controls].values
y = df["revenue"].values

pipe = Pipeline([
    ("scale", ColumnTransformer([
        ("x", StandardScaler(), slice(0,len(channels))),
        ("z", StandardScaler(), slice(len(channels), len(channels)+Z.shape[1])),
    ], remainder="drop")),
    ("ridge", RidgeCV(alphas=np.logspace(-3,3,21), cv=5))
])

XZ = np.hstack([X, Z])
pipe.fit(XZ, y)
coef = pipe.named_steps["ridge"].coef_[:len(channels)]

roi = coef 
pd.DataFrame({"channel":channels, "ROI_baseline_ridge":roi}).round(4)

,channel,ROI_baseline_ridge
0,tv_spend,454.6171
1,search_spend,-46.3875
2,social_spend,413.0915
3,display_spend,-1635.5186


In [8]:
base_spend = df[channels].values
S0 = base_spend.mean(axis=0)

def forward(spend):
    X_opt = []
    for i, c in enumerate(channels):
        xs = adstock(spend[i] * np.ones_like(y), L=8, alpha=0.6)
        xs = saturation(xs, k=np.quantile(xs,0.7)+1e-6, s=0.6)
        X_opt.append(xs)
    X_opt = np.vstack(X_opt).T
    XZ_opt = np.hstack([X_opt, Z])
    return pipe.predict(XZ_opt).sum()

cons = ({'type':'eq','fun': lambda s: s.sum() - S0.sum()},)
bnds = [(0, S0[i]*2.0) for i in range(len(channels))]
res = minimize(lambda s: -forward(s), x0=S0, bounds=bnds, constraints=cons, method="SLSQP")
pd.DataFrame({"channel":channels, "current":S0, "optimized":res.x}).round(2)

,channel,current,optimized
0,tv_spend,9147.24,9147.24
1,search_spend,4380.46,4380.46
2,social_spend,2403.69,2403.69
3,display_spend,1415.61,1415.61


📊 **Regularized Marketing Mix Model (MMM) with Adstock & Saturation**

In this notebook, I implemented a Marketing Mix Model (MMM) using only open-source Python libraries (scikit-learn, numpy, scipy), without relying on external frameworks.

**Key Features:**

* **Adstock Transformation:** Models the carryover effect of marketing spend across time.

* **Saturation Function:** Captures the diminishing returns of spend at higher levels.

* **Regularization (Ridge Regression):** Stabilizes coefficient estimates and reduces overfitting.

* **ROI Estimation:** Calculates channel-level ROI based on the fitted coefficients.

* **Budget Optimization:** Uses scipy.optimize.minimize with constraints to recommend an optimized allocation across channels.

**Why this approach?**
This version is framework-free and fully reproducible in any Python environment. Recruiters and hiring managers can easily understand the math and logic, since everything is implemented explicitly.